In [4]:
import os
from dotenv import load_dotenv

load_dotenv("./credentials.env")

True

In [5]:
from azure.cosmos import exceptions, CosmosClient, PartitionKey

# Create the client to interact with the Azure Cosmos DB resource
client = CosmosClient(os.getenv("COSMOSDB_ENDPOINT"), os.getenv("COSMOSDB_KEY"))

In [6]:
# Create a database in Azure Cosmos DB.
try:
    database = client.create_database_if_not_exists(id="VectorSearchTutorial")
    print(f"Database created: {database.id}")

except exceptions.CosmosResourceExistsError:
    print("Database already exists.")

Database created: VectorSearchTutorial


In [7]:
# Create a container in Azure Cosmos DB.
try:
    partition_key_path = PartitionKey(path="/id")
    container = database.create_container_if_not_exists(
        id="AzureServices",
        partition_key=partition_key_path,
        offer_throughput=400,
    )
    print(f"Container created: {container.id}")

except exceptions.CosmosResourceExistsError:
    print("Container already exists.")

Container created: AzureServices


In [8]:
import json

# Load text-sample.json data file
# https://raw.githubusercontent.com/microsoft/AzureDataRetrievalAugmentedGenerationSamples/main/DataSet/AzureServices/text-sample.json
data_file = open(file="./text-sample.json", mode="r")
#data_file = open(file="../../DataSet/AzureServices/text-sample_w_embeddings.json", mode="r") # load this file instead if embeddings were previously created and saved.
data = json.load(data_file)
data_file.close()

In [9]:
# Create data items for every entry in the dataset, insert them into the database and collection specified above.
for data_item in data:
    try:
        container.create_item(body=data_item)
    
    except exceptions.CosmosResourceExistsError:
        print("Data item already exists.")